In [1]:
from abc import ABCMeta, abstractmethod
from queue import PriorityQueue
import random
import math

# Local Beam Search implementation

<img src="beam.png" width="400"/>

In [2]:
class LocalBeamSearch():
    """
    Local Beam Search implementation

    Methods
    -------
    search(state)
        Runs Local Beam Search starting from a given start state, and returns the solution and the number of expanded states.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """
    __metaclass__ = ABCMeta
    def __init__(self, num_beams):
        """
        Parameters
        ----------
        num_beams
            An integer defining the beam size.
        """
        self.num_beams = int(num_beams)

    def search(self, state):
        """
        Runs Local Beam Search starting from a given start state, and returns the solution and the number of expanded states.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        tuple
            A tuple describing a unique world configuration.
        int
            The number of states expanded during the search.
        """

        num_expanded_states = 0                                   # number of states expanded during the search
        lowest_cost = math.inf

        states_to_expand = PriorityQueue()                        # stack of states to be expanded
        states_to_expand.put((math.inf, state))
        while not states_to_expand.empty():
            states = set()
            current_cost = math.inf
            while len(states) < self.num_beams:                   # get next states to be expanded
                if states_to_expand.empty():
                    break
                cost, state = states_to_expand.get()
                if state not in states:
                    states.add(state)
                    current_cost = min(current_cost, cost)

            num_expanded_states += len(states)                    # add current state to the solution

            for state in states:                                  # if any current state is a goal state, return solution
                if self.goal_test(state):
                    return state, num_expanded_states

            if current_cost != math.inf and current_cost >= lowest_cost:
                break
            lowest_cost = current_cost

            states_to_expand = PriorityQueue()
            for state in states:
                for child, heur_cost in self.successor(state):    # add successors to the stack of states to be expanded
                    if child not in states:
                        states_to_expand.put((heur_cost, child))

        return None, num_expanded_states                          # if no solution is found, return None

    @abstractmethod
    def successor(self, state):
        """
        Finds the list of successors for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        list
            A list of pairs (action,state) with all states that can be reached from the given state with a single action.
        """
        pass

    @abstractmethod
    def goal_test(self, state):
        """
        Checks if the current state is a goal state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        bool
             True if the given state is a goal state, and False otherwise.
        """
        pass

# 8-queens

- Puzzle with chess queens in an $8\times8$ grid. The goal is to place 8 queens so that no queen attacks another.

<img src="8queens.png" width="300"/>

- **Heuristic function**: number of queen pairs attacking each other.

In [3]:
class KQueens(LocalBeamSearch):
    """
    8-queens solution using Local Beam Search.

    Methods
    -------
    show()
        Visualize the current state.
    move(action)
        Apply an action to the current state.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """

    def __init__(self, state=None, k=8, **kargs):
        """
        Parameters
        ----------
        state
            A tuple describing a unique world configuration.
        k
            An integer defining the size of the board and the number of queens.
        """
        super().__init__(kargs['num_beams'] if 'num_beams' in kargs else 1)
        self.k = k
        if state is not None:
            self.state = state
        else:
            self.state = self.__get_random_state()

    def __get_random_state(self):
        """
        Generates a random puzzle configuration.

        Return
        ----------
        tuple
            A tuple describing a unique puzzle configuration.
        """
        return tuple(random.randrange(self.k) for i in range(self.k))

    def show(self):
        """
        Prints the current state.
        """
        print('╔'+'╦'.join(['═══']*self.k)+'╗')
        for i in range(self.k):
            print('║', end=' ')
            for j in range(self.k):
                if self.state[j] == i:
                    print('W', end=' ')
                else:
                    print(' ', end=' ')
                print('║', end=' ')
            print()
            if i < self.k-1:
                print('╠'+'╬'.join(['═══']*self.k)+'╣')
        print('╚'+'╩'.join(['═══']*self.k)+'╝')

    def move(self, action):
        """
        Move the queen to a different row in the same column.

        Parameters
        ----------
        action
            A pair of integers defining column and row, respectively.
        """
        a, b = action
        self.state[a] = b

    def heuristic_function(self, state):
        """
        Computes the heuristic function for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        list
            An integer representing the number of queen pairs that can attack each other.
        """
        cost = 0
        for i in range(self.k):
            for j in range(i+1,self.k):
                if state[i] == state[j] or j-i == abs(state[j]-state[i]):
                    cost += 1
        return cost

    def successor(self, state):
        """
        Finds the list of successors for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        list
            A list of pairs (state,heuristic) with all states that can be reached from the given states with a single move.
        """
        successors = []
        for i in range(self.k):
            child = list(state)
            for j in range(self.k):
                if j != state[i]:
                    child[i] = j
                    successors.append((tuple(child), self.heuristic_function(child)))
        return successors

    def goal_test(self, state):
        """
        Checks if no queen can attack another.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        bool
             True if the given state is a goal state, and False otherwise.
        """

        return self.heuristic_function(state) == 0

In [6]:
puzzle = KQueens(num_beams=1)
print('Start state:')
puzzle.show()

state, num_states = puzzle.search(puzzle.state)
if state is not None:
    puzzle = KQueens(state=state)
    print('Found a solution after expanding {} states!'.format(num_states))
    puzzle.show()
else:
    print('Could not find a solution after expanding {} states!'.format(num_states))

Start state:
╔═══╦═══╦═══╦═══╦═══╦═══╦═══╦═══╗
║   ║ W ║   ║   ║   ║ W ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║ W ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║ W ║ W ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║ W ║   ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║ W ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║ W ║   ║   ║   ║ 
╚═══╩═══╩═══╩═══╩═══╩═══╩═══╩═══╝
Found a solution after expanding 5 states!
╔═══╦═══╦═══╦═══╦═══╦═══╦═══╦═══╗
║   ║ W ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║ W ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║ W ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║ W ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║ W ║   ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬

In [7]:
for num_beams in range(1, 11):

    N = 100

    num_success = 0
    cost_success = 0
    num_failure = 0
    cost_failure = 0

    for i in range(N):
        puzzle = KQueens(num_beams=num_beams)
        state, num_states = puzzle.search(puzzle.state)
        if state is not None:
            num_success += 1
            cost_success += num_states
        else:
            num_failure += 1
            cost_failure += num_states

    print('Number of beams: {}'.format(num_beams))
    print()
    print('Success rate: {:.2f}%'.format(100.0*num_success/N))
    print('Average success cost: {:.2f}'.format((cost_success/num_success if num_success > 0 else -1)))
    print()
    print('Failure rate: {:.2f}%'.format(100.0*num_failure/N))
    print('Average failure cost: {:.2f}'.format((cost_failure/num_failure if num_failure > 0 else -1)))
    print('\n---\n')

Number of beams: 1

Success rate: 12.00%
Average success cost: 4.67

Failure rate: 88.00%
Average failure cost: 5.09

---

Number of beams: 2

Success rate: 32.00%
Average success cost: 9.25

Failure rate: 68.00%
Average failure cost: 9.06

---

Number of beams: 3

Success rate: 40.00%
Average success cost: 12.85

Failure rate: 60.00%
Average failure cost: 12.80

---

Number of beams: 4

Success rate: 37.00%
Average success cost: 17.00

Failure rate: 63.00%
Average failure cost: 16.75

---

Number of beams: 5

Success rate: 43.00%
Average success cost: 20.42

Failure rate: 57.00%
Average failure cost: 21.61

---

Number of beams: 6

Success rate: 40.00%
Average success cost: 25.00

Failure rate: 60.00%
Average failure cost: 25.20

---

Number of beams: 7

Success rate: 45.00%
Average success cost: 29.31

Failure rate: 55.00%
Average failure cost: 27.85

---

Number of beams: 8

Success rate: 51.00%
Average success cost: 33.00

Failure rate: 49.00%
Average failure cost: 33.49

---

Numb